In [1]:
import pandas as pd

In [2]:
df_MB = pd.read_csv('./IntermediateProducts/Results_step_3.csv')

In [3]:
df_NE = pd.read_csv('./SourceData/Final_NE_proteins.csv')

In [20]:
df_MB.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1871 entries, 0 to 2726
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Entry_original   1871 non-null   object
 1   Organism         1871 non-null   object
 2   Gene_name        1871 non-null   object
 3   Protein_name     1871 non-null   object
 4   AH_or_Not        1871 non-null   object
 5   AA_sequence      1871 non-null   object
 6   Prediction       1871 non-null   object
 7   SubCell_Uniprot  1871 non-null   object
 8   Entry_Hs         1871 non-null   object
 9   Entry_Mm         1871 non-null   object
dtypes: object(10)
memory usage: 160.8+ KB


In [18]:
len(df_MB.Entry_Hs.unique())

1871

In [19]:
df_MB = df_MB.drop_duplicates(subset=['Entry_Hs'])

In [21]:
df_MB[df_MB.Entry_Hs == 'Not_found']

,Entry_original,Organism,Gene_name,Protein_name,AH_or_Not,AA_sequence,Prediction,SubCell_Uniprot,Entry_Hs,Entry_Mm
0,Q16928,Anopheles albimanus,W,Protein white,Non-AH,MTINTDDQYADGESKTTISSNRRYSTSSFQDQSMEDDGINATLTND...,0000000000000000000000000000000000000000000000...,Membrane,Not_found,Not_found


In [7]:
df_NE.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 281 entries, 0 to 280
Data columns (total 19 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Unnamed: 0                281 non-null    int64  
 1   UniprotID                 281 non-null    object 
 2   GeneName                  281 non-null    object 
 3   Schirmer_2003             281 non-null    float64
 4   Korfali_2010              281 non-null    float64
 5   Wilkie_2010               281 non-null    float64
 6   Korfali_2012              281 non-null    float64
 7   Korfali_2012_NE:MM-ratio  281 non-null    float64
 8   Cheng_2019                281 non-null    float64
 9   Cheng_2019_Score:Undiff   281 non-null    float64
 10  Cheng_2019_Score:Adipo    281 non-null    float64
 11  Cheng_2019_Score:Myo      281 non-null    float64
 12  #ProteomePapers           281 non-null    float64
 13  HPA                       281 non-null    int64  
 14  HPA_reliab

In [5]:
len(df_MB[df_MB.Entry_Hs != 'Not_found'])

1986

In [22]:
df_merged = df_MB.merge(df_NE, left_on='Entry_Hs', right_on='UniprotID', how='inner')

In [50]:
df_NE[df_NE.GeneName == 'NEMP2']

,Unnamed: 0,UniprotID,GeneName,Schirmer_2003,Korfali_2010,Wilkie_2010,Korfali_2012,Korfali_2012_NE:MM-ratio,Cheng_2019,Cheng_2019_Score:Undiff,Cheng_2019_Score:Adipo,Cheng_2019_Score:Myo,#ProteomePapers,HPA,HPA_reliability,HPA_loc,Results,Uniprot_NEorER,Uniprot_subcell


In [23]:
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 120 entries, 0 to 119
Data columns (total 29 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Entry_original            120 non-null    object 
 1   Organism                  120 non-null    object 
 2   Gene_name                 120 non-null    object 
 3   Protein_name              120 non-null    object 
 4   AH_or_Not                 120 non-null    object 
 5   AA_sequence               120 non-null    object 
 6   Prediction                120 non-null    object 
 7   SubCell_Uniprot           120 non-null    object 
 8   Entry_Hs                  120 non-null    object 
 9   Entry_Mm                  120 non-null    object 
 10  Unnamed: 0                120 non-null    int64  
 11  UniprotID                 120 non-null    object 
 12  GeneName                  120 non-null    object 
 13  Schirmer_2003             120 non-null    float64
 14  Korfali_20

In [10]:
df_merged.head()

,Entry_original,Organism,Gene_name,Protein_name,AH_or_Not,AA_sequence,Prediction,SubCell_Uniprot,Entry_Hs,Entry_Mm,...,Cheng_2019_Score:Undiff,Cheng_2019_Score:Adipo,Cheng_2019_Score:Myo,#ProteomePapers,HPA,HPA_reliability,HPA_loc,Results,Uniprot_NEorER,Uniprot_subcell
0,Q06136,Homo sapiens,KDSR,3-ketodihydrosphingosine reductase,AH,MLLLAAAFLVAFVLLLYMVSPLISPKPLALPGAHVVVTGGSSGIGK...,0000000000000000000000000000000000000000000000...,Endoplasmic reticulum membrane,Q06136,Q6GV12,...,0.253493,0.420695,0.720657,1.0,0,NO,NO,{'entryType': 'UniProtKB reviewed (Swiss-Prot)...,1.0,Endoplasmic reticulum membrane
1,P39656,Homo sapiens,DDOST,Dolichyl-diphosphooligosaccharide--protein gly...,Non-AH,MGYFRCARAGSFGRRRKMEPSTAARAWALFWLLLPLLGAVCASGPR...,0000000000000000000000000000000000000000000000...,Endoplasmic reticulum membrane,P39656,O54734,...,0.322756,0.495222,0.631277,1.0,0,NO,NO,{'entryType': 'UniProtKB reviewed (Swiss-Prot)...,1.0,Endoplasmic reticulum membrane
2,Q9H1E5,Homo sapiens,TMX4,Thioredoxin-related transmembrane protein 4,AH,MAGGRCGPQLTALLAAWIAAVAATAGPEEAALPPEQSRVQPMTASN...,0000000000000000000000000000000000000000000000...,Nucleus inner membrane,Q9H1E5,Q8C0L0,...,0.932352,0.792500,0.844175,4.0,1,Supported,Nuclear membrane: Supported,{'entryType': 'UniProtKB reviewed (Swiss-Prot)...,1.0,"Nucleus inner membrane, Endoplasmic reticulum ..."
3,O75844,Homo sapiens,ZMPSTE24,CAAX prenyl protease 1 homolog,AH,MGMWASLDALWEMPAEKRIFGAVLLFSWTVYLWETFLAQRQRRIYK...,0000000000000000000000000000000000000000000000...,Unknown,O75844,Q80W54,...,0.481677,0.402820,0.625970,1.0,0,NO,NO,{'entryType': 'UniProtKB reviewed (Swiss-Prot)...,1.0,"Endoplasmic reticulum membrane, Nucleus inner ..."
4,Q9BX95,Homo sapiens,SGPP1,Sphingosine-1-phosphate phosphatase 1,AH,MSLRQRLAQLVGRLQDPQKVARFQRLCGVEAPPRRSADRREDEKAE...,1111111111100000000000000000000000000000000000...,Unknown,Q9BX95,Q9JI99,...,0.582937,0.301548,0.758509,1.0,1,Supported,Nucleus: Supported,{'entryType': 'UniProtKB reviewed (Swiss-Prot)...,1.0,"Endoplasmic reticulum membrane, Cell membrane"


In [24]:
df_NE_AH = df_merged[df_merged.AH_or_Not == 'AH']

In [39]:
df_NE_NonAH = df_merged[df_merged.AH_or_Not == 'Non-AH']

In [26]:
len(df_NE_AH)

87

In [27]:
len(df_NE_AH.Entry_Hs.unique())

87

In [28]:
df_NE_AH.head(n=50)

,Entry_original,Organism,Gene_name,Protein_name,AH_or_Not,AA_sequence,Prediction,SubCell_Uniprot,Entry_Hs,Entry_Mm,...,Cheng_2019_Score:Undiff,Cheng_2019_Score:Adipo,Cheng_2019_Score:Myo,#ProteomePapers,HPA,HPA_reliability,HPA_loc,Results,Uniprot_NEorER,Uniprot_subcell
0,Q06136,Homo sapiens,KDSR,3-ketodihydrosphingosine reductase,AH,MLLLAAAFLVAFVLLLYMVSPLISPKPLALPGAHVVVTGGSSGIGK...,0000000000000000000000000000000000000000000000...,Endoplasmic reticulum membrane,Q06136,Q6GV12,...,0.253493,0.420695,0.720657,1.0,0,NO,NO,{'entryType': 'UniProtKB reviewed (Swiss-Prot)...,1.0,Endoplasmic reticulum membrane
2,Q9H1E5,Homo sapiens,TMX4,Thioredoxin-related transmembrane protein 4,AH,MAGGRCGPQLTALLAAWIAAVAATAGPEEAALPPEQSRVQPMTASN...,0000000000000000000000000000000000000000000000...,Nucleus inner membrane,Q9H1E5,Q8C0L0,...,0.932352,0.792500,0.844175,4.0,1,Supported,Nuclear membrane: Supported,{'entryType': 'UniProtKB reviewed (Swiss-Prot)...,1.0,"Nucleus inner membrane, Endoplasmic reticulum ..."
3,O75844,Homo sapiens,ZMPSTE24,CAAX prenyl protease 1 homolog,AH,MGMWASLDALWEMPAEKRIFGAVLLFSWTVYLWETFLAQRQRRIYK...,0000000000000000000000000000000000000000000000...,Unknown,O75844,Q80W54,...,0.481677,0.402820,0.625970,1.0,0,NO,NO,{'entryType': 'UniProtKB reviewed (Swiss-Prot)...,1.0,"Endoplasmic reticulum membrane, Nucleus inner ..."
4,Q9BX95,Homo sapiens,SGPP1,Sphingosine-1-phosphate phosphatase 1,AH,MSLRQRLAQLVGRLQDPQKVARFQRLCGVEAPPRRSADRREDEKAE...,1111111111100000000000000000000000000000000000...,Unknown,Q9BX95,Q9JI99,...,0.582937,0.301548,0.758509,1.0,1,Supported,Nucleus: Supported,{'entryType': 'UniProtKB reviewed (Swiss-Prot)...,1.0,"Endoplasmic reticulum membrane, Cell membrane"
5,Q07065,Homo sapiens,CKAP4,Cytoskeleton-associated protein 4,AH,MPSAKQRGSKGGHGAASPSEKGAHPSGGADDVAKKPPPAPQQPPPP...,0000000000000000000000000000000000000000000000...,Unknown,Q07065,Q8BMK4,...,0.630888,0.251646,0.370579,4.0,0,NO,NO,{'entryType': 'UniProtKB reviewed (Swiss-Prot)...,1.0,"Endoplasmic reticulum membrane, Cell membrane,..."
6,Q8TCG5,Homo sapiens,CPT1C,"Carnitine O-palmitoyltransferase 1, brain isoform",AH,MAEAHQAVGFRPSLTSDGAEVELSAPVLQEIYLSGLRSWKRHLSRF...,0000000000000000000000000000000111111111111111...,Unknown,Q8TCG5,Q8BGD5,...,0.000000,0.788527,0.642311,1.0,0,NO,NO,{'entryType': 'UniProtKB reviewed (Swiss-Prot)...,1.0,"Cell projection, dendrite, Cell projection, ax..."
7,Q6NUQ4,Homo sapiens,TMEM214,Transmembrane protein 214,AH,MATKTAGVGRWEVVKKGRRPGVGAGAGGRGGGRNRRALGEANGVWK...,0000000000000000000000000000000000000000000000...,Endoplasmic reticulum membrane,Q6NUQ4,Q8BM55,...,0.370504,0.648688,0.282569,3.0,0,NO,NO,{'entryType': 'UniProtKB reviewed (Swiss-Prot)...,1.0,Endoplasmic reticulum membrane
8,Q9BUB7,Homo sapiens,TMEM70,"Transmembrane protein 70, mitochondrial",AH,MLFLALGSPWAVELPLCGRRTALCAAAALRGPRASVSRASSSSGPS...,0000000000000000000000000000000000000000000000...,Mitochondrion inner membrane,Q9BUB7,Q921N7,...,0.000000,0.000000,0.000000,3.0,1,Supported,Nucleoplasm: Supported;Mitochondria: Supported,{'entryType': 'UniProtKB reviewed (Swiss-Prot)...,0.0,Mitochondrion inner membrane
10,A0FGR8,Homo sapiens,ESYT2,Extended synaptotagmin-2,AH,MTANRDAALSSHRHPGCAQRPRTPTFASSSQRRSAFGFDDGNFPGL...,0000000000000000000000000000000000000000000000...,Unknown,A0FGR8,Q3TZZ7,...,0.000000,0.000000,0.000000,1.0,0,NO,NO,{'entryType': 'UniProtKB reviewed (Swiss-Prot)...,1.0,"Cell membrane, Endoplasmic reticulum membrane"
12,Q8NC44,Homo sapiens,RETREG2,Reticulophagy regulator 2,AH,MASGGGGGNTGAGGGPGMGLSLGLGLGLSLGMSEATSEAEEEAATA...,0000000000000000000000000000000000000000000000...,Membrane,Q8NC44,Q6NS82,...,0.000000,0.000000,0.000000,1.0,0,NO,NO,{'entryType': 'UniProtKB reviewed (Swiss-Prot)...,1.0,Endoplasmic reticulum membrane


In [29]:
for i in [1,2,3,4,5]:
    print(len(df_NE_AH[df_NE_AH['#ProteomePapers'] == i]))

48
10
12
15
2


In [48]:
df_NE_AH[df_NE_AH['#ProteomePapers'] == 3]

,Entry_original,Organism,Gene_name,Protein_name,AH_or_Not,AA_sequence,Prediction,SubCell_Uniprot,Entry_Hs,Entry_Mm,...,Cheng_2019_Score:Undiff,Cheng_2019_Score:Adipo,Cheng_2019_Score:Myo,#ProteomePapers,HPA,HPA_reliability,HPA_loc,Results,Uniprot_NEorER,Uniprot_subcell
7,Q6NUQ4,Homo sapiens,TMEM214,Transmembrane protein 214,AH,MATKTAGVGRWEVVKKGRRPGVGAGAGGRGGGRNRRALGEANGVWK...,0000000000000000000000000000000000000000000000...,Endoplasmic reticulum membrane,Q6NUQ4,Q8BM55,...,0.370504,0.648688,0.282569,3.0,0,NO,NO,{'entryType': 'UniProtKB reviewed (Swiss-Prot)...,1.0,Endoplasmic reticulum membrane
8,Q9BUB7,Homo sapiens,TMEM70,"Transmembrane protein 70, mitochondrial",AH,MLFLALGSPWAVELPLCGRRTALCAAAALRGPRASVSRASSSSGPS...,0000000000000000000000000000000000000000000000...,Mitochondrion inner membrane,Q9BUB7,Q921N7,...,0.000000,0.000000,0.000000,3.0,1,Supported,Nucleoplasm: Supported;Mitochondria: Supported,{'entryType': 'UniProtKB reviewed (Swiss-Prot)...,0.0,Mitochondrion inner membrane
13,Q9NX78,Homo sapiens,TMEM260,Transmembrane protein 260,AH,MSPHGDGRGQAQGRAVRVGLRRSGGIRGGVAVFAAVAAVFTFTLPP...,0000000000000000000000000000000000000000000000...,Membrane,Q9NX78,Q8BMD6,...,0.000000,0.000000,0.000000,3.0,0,NO,NO,{'entryType': 'UniProtKB reviewed (Swiss-Prot)...,1.0,"Endoplasmic reticulum membrane, Membrane"
19,Q5SNT2,Homo sapiens,TMEM201,Transmembrane protein 201,AH,MEGVSALLARCPTAGLAGGLGVTACAAAGVLLYRIARRMKPTHTMV...,0000000000000000000000000000000011111110000000...,Unknown,Q5SNT2,A2A8U2,...,0.000000,0.000000,0.000000,3.0,1,Supported,Nucleus: Supported;Nuclear membrane: Supported,{'entryType': 'UniProtKB reviewed (Swiss-Prot)...,1.0,"Nucleus inner membrane, Cytoplasm, cytoskeleto..."
34,Q9H330,Homo sapiens,TMEM245,Transmembrane protein 245,AH,MADGGGPKDAPSLRSSPGPAPRVPRAVGPSGGGGETPRTAALALRF...,0000000000000000000000000000000000000000000000...,Membrane,Q9H330,B1AZA5,...,0.000000,0.000000,0.000000,3.0,0,NO,NO,{'entryType': 'UniProtKB reviewed (Swiss-Prot)...,0.0,Membrane
60,A7MBC7,Bos taurus,NEMP1,Nuclear envelope integral membrane protein 1,AH,MAGGMKVAVLPAVGAGPWSWGAGGCGAVRLLLVLFGCFVCGSAGID...,0000000000000000000000000000000000000000000000...,Unknown,O14524,Q6ZQE4,...,0.000000,0.000000,0.000000,3.0,0,NO,NO,{'entryType': 'UniProtKB reviewed (Swiss-Prot)...,1.0,"Nucleus inner membrane, Nucleus envelope"
72,Q6P752,Rattus norvegicus,Tor1aip2,Torsin-1A-interacting protein 2,AH,MSQTLKSQDTNMSDSGYRDPVEDSQNVLGNDPSVNSQAQDPIVTPS...,0000000000000000000000000000000000000000000000...,Unknown,Q8NFQ8,Q8BYU6,...,0.000000,0.000000,0.000000,3.0,0,NO,NO,{'entryType': 'UniProtKB reviewed (Swiss-Prot)...,0.0,"Endoplasmic reticulum membrane, Nucleus membrane"
75,Q5R589,Pongo abelii,ERG28,Ergosterol biosynthetic protein 28 homolog,AH,MSRFLNVLRSWLVMVSIIAMGNTLQSFRDHTFLYEKLYTGKPNLVN...,0001111110000000000000000000000000000000000000...,Endoplasmic reticulum membrane,Q9UKR5,Q9ERY9,...,0.000000,0.000000,0.000000,3.0,0,NO,NO,{'entryType': 'UniProtKB reviewed (Swiss-Prot)...,1.0,Endoplasmic reticulum membrane
90,Q6GNM0,Xenopus laevis,nrm,Nurim,AH,MSANVQVSGQLSSGPSLPACIVLSAVSLLCFVAGFGTGAEFVRFLS...,0000000000000000000000000000000000000000000000...,Nucleus inner membrane,Q8IXM6,Q8VC65,...,0.000000,0.000000,0.000000,3.0,1,Validated,Nuclear membrane: Validated,{'entryType': 'UniProtKB reviewed (Swiss-Prot)...,1.0,Nucleus inner membrane
96,Q5M7E0,Xenopus laevis,fam210a,Protein FAM210A,AH,MHLLRTLLLRSNTSNISLLTKCSFRASPLHKWPISLRSGSQISLLP...,0000000000000000000000000000000000000000000000...,Unknown,Q96ND0,Q8BGY7,...,0.502097,0.313363,0.182780,3.0,0,NO,NO,{'entryType': 'UniProtKB reviewed (Swiss-Prot)...,0.0,"Membrane, Mitochondrion, Cytoplasm"


In [40]:
for i in [1,2,3,4,5]:
    print(len(df_NE_NonAH[df_NE_NonAH['#ProteomePapers'] == i]))

21
6
3
2
1
